In [2]:
import json
import re
path_list_train_cv = "allcat_train_cv_json_list_inside.txt"
path_list_test = "allcat_test_json_list_inside.txt"
from tqdm import tqdm

In [3]:
def load_text_from_json_object(json_object):
    text = ""
    for i in json_object["paragraphs"]:
        text+=i["text"]+" "#appending a space at end so that there is a gap when next sentence is added
    return text

In [4]:
def clean_text(text):
    '''
    for removing special characters, lowering the case and removing 
    punctuations
    
    params:
        text: str - text string of the document to be cleaned 
    returns:
        text_list : list of cleaned text that has length one. 
                    tfidf_vectoriser accepts only list of strings
    '''
    pattern = r'[^A-Za-z ]+'
    clean_text = [re.sub(pattern,'', str(x.lower())) for x in text.split(" ")]
#     text_list = [" ".join(clean_text)]
    text_list = " ".join(clean_text)
    #if you are sending this to the model,
    #send it as a list, Since Model takes list of text
    return text_list

In [5]:
def find_headings_and_para(file):
    '''
    file: json object
    It checks all paragraphs and filters those will length less than 5
    '''
    n_headings = 0
    n_paragraphs = 0
    headings = []
    for para in file["paragraphs"]:
        if para["is_valid"]:
            if len(para["text"]) > 0:
                if len(para["text"].split(" ")) < 5:
                    n_headings+=1
                    clean_heading = clean_text(para["text"])
                    headings.append(clean_heading)
                else:
                    count=0
                    for run in para["runs"]:
                        count+=1
                        if (run["all_caps"] or run["bold"]) and len(run["text"]) > 6:
                            clean_heading = clean_text(run["text"])
                            headings.append(clean_heading)
                        if count == 2:#checking only for first two runs. As that's where headings are more probable
                            break 
                            
    return n_headings, headings, n_paragraphs
            

In [15]:
!rm data_json_space_corrected_with_tables_try_inline_headings.csv

In [16]:
f_result = open("data_json_space_corrected_with_tables_try_inline_headings.csv", "a+")
f_result.write('doc_name,doc_category,domain, status,word_count,n_tables,n_paragraphs,n_images,n_headings,heading_text,text\n')
with open("raw_json_list.txt") as f:
    for i in tqdm(f):
        path= i.rstrip("\n")
#         print(path)
#         path = "./Journal_Articles/Physical Sciences/Original/ADDMH_2_input.json"
        _,doc_category,domain,status,name = path.split("/")
#         print(doc_category,domain,status,name)
        file = json.load(open(path))
#         print(file.keys())
#         print(file["paragraphs"][0].keys())
        if status == "Original" and file["status"]:
            n_tables = len(file["tables"])
            n_images = file["images_count"]
            word_count = file["word_count"]
            n_tables = len(file["tables"])
            n_headings, headings, n_paragraphs = find_headings_and_para(file)
            text = clean_text(load_text_from_json_object(file))
            heading_text = " ".join(headings)
            result = f'{name},{doc_category},{domain},{status},{word_count},{n_tables},{n_paragraphs},{n_images},{n_headings},{heading_text},{text}\n'
            f_result.write(result)
#             print(text)
f_result.close()

2792it [01:25, 32.68it/s] 
